<a href="https://colab.research.google.com/github/thomaslu678/gee-test/blob/main/clean/7_fetch_met_data_linreg_tif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NOTE: Requires export_clean.csv (cleaned up export CSV with sorts and time columns)

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from datetime import timedelta
import scipy.stats as stats
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize
import geopandas as gpd
from shapely.geometry import Point
import requests

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read cleaned up export csv

In [13]:
calculations_df = pd.read_csv('/content/drive/MyDrive/Year 2/Fall 2025/HONOR 3700/Data/2/export_clean_calculations.csv')

In [14]:
calculations_df

,point_id,time,LST_C,NDVI,NDBI,NDMI,Albedo
0,2505,1988-12-24 01:43:00,-0.527053,0.008107,-0.042450,0.042450,0.185196
1,2527,1988-12-24 01:43:00,-0.527053,0.008107,-0.105622,0.105622,0.183640
2,2528,1988-12-24 01:43:00,-0.527053,-0.011292,-0.099890,0.099890,0.176036
3,2550,1988-12-24 01:43:00,-0.431348,-0.029073,-0.068238,0.068238,0.171996
4,2551,1988-12-24 01:43:00,-0.431348,0.008107,-0.092413,0.092413,0.181680
...,...,...,...,...,...,...,...
2646549,4618,2026-01-31 02:11:17,1.397293,0.192808,0.070690,-0.070690,0.085628
2646550,4619,2026-01-31 02:11:17,1.356276,0.306176,0.073418,-0.073418,0.127741
2646551,4620,2026-01-31 02:11:17,1.575030,0.157747,-0.135409,0.135409,0.149991
2646552,4627,2026-01-31 02:11:17,1.438309,0.157339,0.031072,-0.031072,0.071202


In [161]:
input_df = pd.read_csv('/content/drive/MyDrive/Year 2/Fall 2025/HONOR 3700/Data/2/points.csv')

In [18]:
input_df

,point_id,long,lat,distance
0,1,126.974400,37.570275,321.262762
1,2,126.974402,37.570005,311.450539
2,3,126.974403,37.569735,304.291166
3,4,126.974405,37.569464,299.974648
4,5,126.974407,37.569194,298.624287
...,...,...,...,...
4623,4624,127.044710,37.572190,309.046080
4624,4625,127.044712,37.571920,308.199775
4625,4626,127.044713,37.571649,310.265569
4626,4627,127.044715,37.571379,315.186207


In [23]:
rep_lat = input_df['lat'].mean()
rep_long = input_df['long'].mean()

In [24]:
rep_lat, rep_long

(np.float64(37.56996553367545), np.float64(127.00993605345722))

# Fetch met data

In [27]:
hours = (
    calculations_df['time']
    .dt.floor('H')
    .drop_duplicates()
    .sort_values()
)

hours_str = hours.dt.strftime('%Y-%m-%dT%H:%M')

/tmp/ipython-input-2803264291.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .dt.floor('H')


In [28]:
hours_str

,time
77212,1984-07-30T01:00
22645,1984-08-08T01:00
24685,1984-09-25T01:00
81720,1984-12-05T01:00
83001,1985-02-07T01:00
...,...
2451905,2025-12-22T02:00
2641777,2025-12-30T02:00
2454231,2026-01-07T02:00
2456046,2026-01-23T02:00


In [31]:
def fetch_open_meteo(lat, lon, start, end):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "temperature_2m",
        "start_date": start.strftime('%Y-%m-%d'),
        "end_date": end.strftime('%Y-%m-%d'),
        "timezone": "UTC"
    }
    r = requests.get(url, params=params)
    r.raise_for_status()
    return r.json()

In [36]:
temps = []

for year, group in hours.groupby(hours.dt.year):
    start = group.min()
    end   = group.max()

    data = fetch_open_meteo(rep_lat, rep_long, start, end)

    df = pd.DataFrame({
        "hour": pd.to_datetime(data["hourly"]["time"], utc=True),
        "temperature_C": data["hourly"]["temperature_2m"]
    })

    temps.append(df)

hourly_temp = pd.concat(temps).drop_duplicates(subset="hour")

In [39]:
calculations_df['hour'] = calculations_df['time'].dt.floor('H')

calculations_df = calculations_df.merge(
    hourly_temp,
    on='hour',
    how='left'
)

/tmp/ipython-input-4154394922.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  calculations_df['hour'] = calculations_df['time'].dt.floor('H')


In [41]:
calculations_df['delta_C'] = calculations_df['temperature_C'] - calculations_df['LST_C']

In [127]:
calculations_df

,point_id,time,LST_C,NDVI,NDBI,NDMI,Albedo,hour,temperature_C,delta_C
0,2505,1988-12-24 01:43:00+00:00,-0.527053,0.008107,-0.042450,0.042450,0.185196,1988-12-24 01:00:00+00:00,2.7,3.227053
1,2527,1988-12-24 01:43:00+00:00,-0.527053,0.008107,-0.105622,0.105622,0.183640,1988-12-24 01:00:00+00:00,2.7,3.227053
2,2528,1988-12-24 01:43:00+00:00,-0.527053,-0.011292,-0.099890,0.099890,0.176036,1988-12-24 01:00:00+00:00,2.7,3.227053
3,2550,1988-12-24 01:43:00+00:00,-0.431348,-0.029073,-0.068238,0.068238,0.171996,1988-12-24 01:00:00+00:00,2.7,3.131348
4,2551,1988-12-24 01:43:00+00:00,-0.431348,0.008107,-0.092413,0.092413,0.181680,1988-12-24 01:00:00+00:00,2.7,3.131348
...,...,...,...,...,...,...,...,...,...,...
2646549,4618,2026-01-31 02:11:17+00:00,1.397293,0.192808,0.070690,-0.070690,0.085628,2026-01-31 02:00:00+00:00,-5.6,-6.997293
2646550,4619,2026-01-31 02:11:17+00:00,1.356276,0.306176,0.073418,-0.073418,0.127741,2026-01-31 02:00:00+00:00,-5.6,-6.956276
2646551,4620,2026-01-31 02:11:17+00:00,1.575030,0.157747,-0.135409,0.135409,0.149991,2026-01-31 02:00:00+00:00,-5.6,-7.175030
2646552,4627,2026-01-31 02:11:17+00:00,1.438309,0.157339,0.031072,-0.031072,0.071202,2026-01-31 02:00:00+00:00,-5.6,-7.038309


In [179]:
print(calculations_df['time'].unique())

<DatetimeArray>
['1988-12-24 01:43:00+00:00', '1989-02-26 01:43:56+00:00',
 '1989-03-14 01:43:50+00:00', '1989-05-17 01:44:21+00:00',
 '1989-08-05 01:44:33+00:00', '1991-11-15 01:21:41+00:00',
 '1991-12-01 01:20:53+00:00', '1984-08-08 01:34:24+00:00',
 '1984-09-25 01:34:59+00:00', '1986-06-11 01:28:45+00:00',
 ...
 '2025-06-05 02:10:27+00:00', '2025-07-23 02:10:54+00:00',
 '2025-08-08 02:11:03+00:00', '2025-08-24 02:11:10+00:00',
 '2025-10-27 02:11:21+00:00', '2025-11-12 02:11:26+00:00',
 '2025-11-28 02:11:24+00:00', '2025-12-14 02:11:24+00:00',
 '2025-12-30 02:11:28+00:00', '2026-01-31 02:11:17+00:00']
Length: 970, dtype: datetime64[ns, UTC]


In [153]:
input_df

,point_id,long,lat,distance,m_x,m_y
0,1,126.974400,37.570275,321.262762,2.280000,0.060040
1,2,126.974402,37.570005,311.450539,2.209724,0.058189
2,3,126.974403,37.569735,304.291166,1.231650,0.032433
3,4,126.974405,37.569464,299.974648,0.699075,0.018409
4,5,126.974407,37.569194,298.624287,0.132117,0.003479
...,...,...,...,...,...,...
4623,4624,127.044710,37.572190,309.046080,1.247938,0.032862
4624,4625,127.044712,37.571920,308.199775,0.119830,0.003156
4625,4626,127.044713,37.571649,310.265569,-1.090684,-0.028721
4626,4627,127.044715,37.571379,315.186207,-1.178101,-0.031023


# Fit linear regression

In [162]:
seconds_per_year = 365.2425 * 24 * 3600

calculations_df['time_years'] = (
    calculations_df['time'].astype('int64') / 1e9
) / seconds_per_year

In [163]:
def compute_slope(group):
    # Need at least 2 points to fit a line
    if len(group) < 2:
        return np.nan

    x = group['time_years'].values
    y = group['delta_C'].values

    m, b = np.polyfit(x, y, 1)
    return m

In [164]:
m_df = (
    calculations_df
    .groupby('point_id', as_index=False)
    .apply(compute_slope)
    .rename(columns={None: 'm'})
)

/tmp/ipython-input-3052817466.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_slope)


In [165]:
input_df = input_df.merge(
    m_df,
    on='point_id',
    how='left'
)

In [166]:
input_df

,point_id,long,lat,distance,m
0,1,126.974400,37.570275,321.262762,0.060040
1,2,126.974402,37.570005,311.450539,0.058189
2,3,126.974403,37.569735,304.291166,0.032433
3,4,126.974405,37.569464,299.974648,0.018409
4,5,126.974407,37.569194,298.624287,0.003479
...,...,...,...,...,...
4623,4624,127.044710,37.572190,309.046080,0.032862
4624,4625,127.044712,37.571920,308.199775,0.003156
4625,4626,127.044713,37.571649,310.265569,-0.028721
4626,4627,127.044715,37.571379,315.186207,-0.031023


# Export to .tif

In [142]:
input_df['m'] = pd.to_numeric(input_df['m'], errors='coerce')

In [167]:
gdf = gpd.GeoDataFrame(
    input_df,
    geometry=gpd.points_from_xy(
        input_df['long'],
        input_df['lat']
    ),
    crs="EPSG:4326"
)

gdf_5179 = gdf.to_crs("EPSG:5179")

In [171]:
gdf_5179['m'].isna().sum()

np.int64(0)

In [175]:
pixel_size = 30

xs = gdf_5179.geometry.x.values
ys = gdf_5179.geometry.y.values

xmin = xs.min() - pixel_size / 2
ymax = ys.max() + pixel_size / 2

width = int((xs.max() - xs.min()) / pixel_size) + 1
height = int((ys.max() - ys.min()) / pixel_size) + 1

transform = from_origin(
    xmin,
    ymax,
    pixel_size,
    pixel_size
)

In [176]:
shapes = (
    (geom, value)
    for geom, value in zip(gdf_5179.geometry, gdf_5179['m'])
)

raster = rasterize(
    shapes=shapes,
    out_shape=(height, width),
    transform=transform,
    fill=np.nan,
    dtype="float32"
)

In [177]:
path = "/content/sample_data/input_df_scaled.tif"

with rasterio.open(
    path,
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,
    dtype="float32",
    crs="EPSG:5179",
    transform=transform,
    nodata=np.nan
) as dst:
    dst.write(raster, 1)

# Get final dataset

In [180]:
merged_df = pd.merge(calculations_df, input_df, on="point_id")

In [188]:
merged_df['hour_of_day'] = merged_df['time'].dt.hour

In [191]:
merged_df['month'] = merged_df['time'].dt.month

In [192]:
merged_df

,point_id,time,LST_C,NDVI,NDBI,NDMI,Albedo,hour,temperature_C,delta_C,time_num,time_years,long,lat,distance,m,hour_of_day,month
0,2505,1988-12-24 01:43:00+00:00,-0.527053,0.008107,-0.042450,0.042450,0.185196,1988-12-24 01:00:00+00:00,2.7,3.227053,5.989310e+08,18.979367,127.013452,37.572606,317.553292,-0.029627,1,12
1,2527,1988-12-24 01:43:00+00:00,-0.527053,0.008107,-0.105622,0.105622,0.183640,1988-12-24 01:00:00+00:00,2.7,3.227053,5.989310e+08,18.979367,127.013792,37.572608,317.478476,-0.003934,1,12
2,2528,1988-12-24 01:43:00+00:00,-0.527053,-0.011292,-0.099890,0.099890,0.176036,1988-12-24 01:00:00+00:00,2.7,3.227053,5.989310e+08,18.979367,127.013794,37.572337,287.478569,-0.007349,1,12
3,2550,1988-12-24 01:43:00+00:00,-0.431348,-0.029073,-0.068238,0.068238,0.171996,1988-12-24 01:00:00+00:00,2.7,3.131348,5.989310e+08,18.979367,127.014133,37.572339,287.403753,-0.007378,1,12
4,2551,1988-12-24 01:43:00+00:00,-0.431348,0.008107,-0.092413,0.092413,0.181680,1988-12-24 01:00:00+00:00,2.7,3.131348,5.989310e+08,18.979367,127.014135,37.572068,257.403847,-0.019299,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2646549,4618,2026-01-31 02:11:17+00:00,1.397293,0.192808,0.070690,-0.070690,0.085628,2026-01-31 02:00:00+00:00,-5.6,-6.997293,1.769825e+09,56.083537,127.044375,37.571377,285.926939,-0.037815,2,1
2646550,4619,2026-01-31 02:11:17+00:00,1.356276,0.306176,0.073418,-0.073418,0.127741,2026-01-31 02:00:00+00:00,-5.6,-6.956276,1.769825e+09,56.083537,127.044377,37.571107,294.332867,-0.026298,2,1
2646551,4620,2026-01-31 02:11:17+00:00,1.575030,0.157747,-0.135409,0.135409,0.149991,2026-01-31 02:00:00+00:00,-5.6,-7.175030,1.769825e+09,56.083537,127.044379,37.570837,305.465969,-0.020183,2,1
2646552,4627,2026-01-31 02:11:17+00:00,1.438309,0.157339,0.031072,-0.031072,0.071202,2026-01-31 02:00:00+00:00,-5.6,-7.038309,1.769825e+09,56.083537,127.044715,37.571379,315.186207,-0.031023,2,1


In [193]:
merged_df.columns

Index(['point_id', 'time', 'LST_C', 'NDVI', 'NDBI', 'NDMI', 'Albedo', 'hour',
       'temperature_C', 'delta_C', 'time_num', 'time_years', 'long', 'lat',
       'distance', 'm', 'hour_of_day', 'month'],
      dtype='object')

In [194]:
columns = ['NDVI', 'NDBI', 'NDMI', 'Albedo', 'time_years', 'long', 'lat',
       'distance', 'hour_of_day', 'month', 'm']

In [195]:
export_df = merged_df[columns]

In [196]:
export_df

,NDVI,NDBI,NDMI,Albedo,time_years,long,lat,distance,hour_of_day,month,m
0,0.008107,-0.042450,0.042450,0.185196,18.979367,127.013452,37.572606,317.553292,1,12,-0.029627
1,0.008107,-0.105622,0.105622,0.183640,18.979367,127.013792,37.572608,317.478476,1,12,-0.003934
2,-0.011292,-0.099890,0.099890,0.176036,18.979367,127.013794,37.572337,287.478569,1,12,-0.007349
3,-0.029073,-0.068238,0.068238,0.171996,18.979367,127.014133,37.572339,287.403753,1,12,-0.007378
4,0.008107,-0.092413,0.092413,0.181680,18.979367,127.014135,37.572068,257.403847,1,12,-0.019299
...,...,...,...,...,...,...,...,...,...,...,...
2646549,0.192808,0.070690,-0.070690,0.085628,56.083537,127.044375,37.571377,285.926939,2,1,-0.037815
2646550,0.306176,0.073418,-0.073418,0.127741,56.083537,127.044377,37.571107,294.332867,2,1,-0.026298
2646551,0.157747,-0.135409,0.135409,0.149991,56.083537,127.044379,37.570837,305.465969,2,1,-0.020183
2646552,0.157339,0.031072,-0.031072,0.071202,56.083537,127.044715,37.571379,315.186207,2,1,-0.031023


In [199]:
export_df.columns

Index(['NDVI', 'NDBI', 'NDMI', 'Albedo', 'time_years', 'long', 'lat',
       'distance', 'hour_of_day', 'month', 'm'],
      dtype='object')

In [198]:
export_df.to_csv('dataset.csv', index=False)